In [1]:
import pandas as pd
import numpy as np

from simulator import *

from IPython.display import display

from bokeh.palettes import Spectral11
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
output_notebook(hide_banner=True)

In [2]:
def get_dataset1():
    dataset = []
    for i in [1,2,3]:
        df = pd.read_csv(f'../data/table{i}.csv', dtype={'Genotype': str}).set_index('Genotype').T
        df = df.rename_axis('Label')
        dataset.append(df)
    return dataset
dataset1 = get_dataset1()
#for table in dataset1:
#    display(table)

In [3]:
def get_dataset2():
    df = pd.read_csv('../data/barlow.csv', index_col=0)
    df = df.sort_index(axis=1)
    df = df.rename_axis('Genotype', axis='columns')
    return df
dataset2 = get_dataset2()
#dataset2

In [4]:
def genotype_strings(num):
    return(["{0:0>4b}".format(n) for n in range(num)])

def plot_simulation_bokeh(results, genotypes_to_plot=[]):
    if not genotypes_to_plot:
        genotypes_to_plot = genotype_strings(len(results))
    timesteps = len(next(iter(results.values())))
    #           length of an arbitrary entry from dictionary
    results["index"] = np.array(range(0, timesteps), np.int64)
    source = ColumnDataSource(results)

    TOOLTIPS = [
        ("genotype", "$name"),
        ("abundance", "$y")
    ]
    p = figure(plot_width=600, plot_height=400, y_axis_type="log", y_range=[10**0, 10**10], tooltips=TOOLTIPS)
    p.hover.mode = "vline"
    #genotype_range = ["0000", "0010", "0110", "1110"]
    for genotype, color in zip(genotypes_to_plot, Spectral11):
        p.line("index", genotype, legend=dict(value=genotype), name=genotype, color=color, line_width=1.5, source=source)
    p.legend.click_policy="hide"
    show(p)

In [5]:
landscape = dataset1[2].loc['53.60μM'].tolist()
results = simulate(landscape, timesteps=2000)
plot_simulation_bokeh(results, ["0000", "0010", "0110", "1110"])